# RFSoC-PYNQ Multi-Tile Sync Acquisition Design
---

This overlay demonstrates the capability of the RFSoC4x2 to behave as an arbitrary waveofmr generator (AWG).
<p align="center">
  <img src="https://www.realdigital.org/img/b228828ef9555df47b6dc0bca8e38f90.png" width="28%" />
</p>

## Board Silkscreen Mappings: 
* DAC A -> DAC Tile 230 Channel 0
* DAC B -> DAC Tile 228 Channel 0

## Design Overview
This overlay uses two DAC waveform generator that are set to act in the first an second Nyquist zones spesifically. Both DACs operate at 8GSps. Independent waveforms can be uploaded to the DACs to allow for AWG generation of signals up to 8GHz

<div class="alert alert-block alert-danger">Because of the clocking strategy used in this overlay it is best run after first power-cycling your board.</div>

In [ ]:
from rfsoc_awg import awgOverlay
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ol = awgOverlay('awgStable.bit',"stable") 
#ol = awgOverlay('awgDev.bit',"dev")

# Generate Waveforms for Loopback
 This overlay uses a single waveform memory that is broadcast to multiple DAC tiles.
### Prepare the DAC Memory
The cell below generates examples waveforms and has customizable paramters.  For example, one can adjust the center frequency, Fc, and re-generate a sinewave.

In [ ]:
DAC_SR = 8.0E9  # Sample rate of DACs and ADCs is 4.0 GHz
Fc = ol.frequency_round(0.5E9,DAC_SR) # Set center frequency of waveform to 250.0 MHz
DAC_Amplitude = 16383  # 14bit DAC +16383/-16384
X_axis =  (1/DAC_SR)*np.arange(0,ol.dac_player.shape[0]) #generate a buffer the exact length of the memory at time intervals that line up with the sample rate

print(ol.dac_player.shape[0])
print(Fc)

1048576
500000000.0


In [ ]:
# Generating a frequency comb

frequencyList = np.arange(0.1*10**9,4.0*10**9,50*10**6)
sineTemplate=np.zeros(len(X_axis))
frquencyNum=len(sineTemplate)
combCounter=1
for f in frequencyList:
    sineTemplate = sineTemplate+(1/len(frequencyList))*np.sin(2*np.pi*ol.frequency_round(f,DAC_SR)*X_axis) #frequency round ensures the frequency is an integer multiple of the dac buffer frequency resolution, ensuring that there is no jump when it loops around
    print(f"Completed {combCounter} of {frquencyNum} frequency peaks")
    combCounter+=1
sineTemplate = DAC_Amplitude*sineTemplate

print(f"There are {frquencyNum} peaks in the frequency comb")
DAC_sinewave = DAC_Amplitude*np.sin(2*np.pi*Fc*X_axis)
DAC_DC = DAC_Amplitude*np.ones(len(X_axis))

58


In [ ]:
#play frequency, player is used for first nyquist zone, player2 for second, however due to aliassing your signal
#will still be present in both, the difference between the 2 is the power distribution across first and second nyquist
ol.dac_player[:]=np.int16(sineTemplate)
#ol.dac_player2[:]=np.int16(sineTemplate) 
ol.trigger_dac() #turns on both dacs, can be turned off using ol.dac_enable.off()

2870002746.5820312
2870010375.9765625
-7629.39453125


---